# FESTA Framework Demo: LVLM Consistency Assessment

**Paper**: [FESTA: Towards understanding the failure points in LVLMs](https://arxiv.org/pdf/2410.23499.pdf)

**GitHub**: https://github.com/iiscleap/mllm-uncertainty-estimation/

This notebook demonstrates the FESTA framework's assessment of Large Vision-Language Models (LVLMs) consistency using **real examples and failure cases from the BLINK dataset**.

FESTA evaluates two types of consistency:
1. **Equivalent Input Assessment**: Same semantic meaning, different phrasing
2. **Complementary Input Assessment**: Opposite/negated meanings requiring different responses

## Setup and Dependencies

In [ ]:
# Install required packages
!pip install torch torchvision transformers pillow requests accelerate bitsandbytes
!pip install git+https://github.com/LLaVA-VL/LLaVA-NeXT.git

In [ ]:
import torch
import json
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration
import warnings
warnings.filterwarnings('ignore')

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

## Load LVLM Model (LLaVA-NeXT)

In [ ]:
# Load LLaVA-NeXT model
model_id = "llava-hf/llava-v1.6-mistral-7b-hf"

processor = LlavaNextProcessor.from_pretrained(model_id)
model = LlavaNextForConditionalGeneration.from_pretrained(
    model_id, 
    torch_dtype=torch.float16, 
    low_cpu_mem_usage=True,
    device_map="auto"
)

print("LLaVA-NeXT model loaded successfully!")

In [ ]:
def generate_response(image, question, max_length=100):
    """
    Generate model response for image-question pair
    """
    prompt = f"USER: <image>\n{question}\nASSISTANT:"
    
    inputs = processor(prompt, image, return_tensors="pt").to(device)
    
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=max_length,
            do_sample=True,
            temperature=0.1,
            pad_token_id=processor.tokenizer.eos_token_id
        )
    
    # Decode and clean response
    response = processor.decode(output[0], skip_special_tokens=True)
    # Extract only the assistant's response
    response = response.split("ASSISTANT:")[-1].strip()
    
    return response

## Load BLINK Dataset Examples

These examples are from the actual BLINK dataset, including real failure cases identified in LLaVA responses.

In [ ]:
# Download example images and configuration
import os
import urllib.request

# GitHub raw URL base
base_url = "https://raw.githubusercontent.com/iiscleap/mllm-uncertainty-estimation/main/examples/"

# Create local directory
os.makedirs("examples", exist_ok=True)

# Download configuration
config_url = f"{base_url}demo_examples.json"
urllib.request.urlretrieve(config_url, "examples/demo_examples.json")

# Load example configuration
with open("examples/demo_examples.json", "r") as f:
    examples = json.load(f)

print("Loaded BLINK dataset examples:")
print(f"- Equivalent examples: {len(examples['equivalent_examples'])}")
print(f"- Complementary examples: {len(examples['complementary_examples'])}")

In [ ]:
# Download example images
image_files = [
    "val_Spatial_Relation_1.jpg",
    "val_Spatial_Relation_1_contrast1.jpg", 
    "val_Spatial_Relation_1_masking1.jpg",
    "val_Spatial_Relation_1_negated_contrast1.jpg"
]

for img_file in image_files:
    img_url = f"{base_url}{img_file}"
    urllib.request.urlretrieve(img_url, f"examples/{img_file}")
    print(f"Downloaded: {img_file}")

print("\nAll images downloaded successfully!")

## FESTA Assessment: Equivalent Input Consistency

**Equivalent inputs** have the same semantic meaning but different phrasing. A consistent model should provide similar responses.

In [ ]:
def display_equivalent_assessment(example_id):
    # Find the example
    example = next(ex for ex in examples['equivalent_examples'] if ex['id'] == example_id)
    
    # Load images
    orig_img = Image.open(f"examples/{example['original_image']}")
    pert_img = Image.open(f"examples/{example['perturbed_image']}")
    
    # Generate responses
    print(f"\n🔍 EQUIVALENT INPUT ASSESSMENT: {example['perturbation_type'].upper()}")
    print(f"Expected failure: {'YES' if example['expected_failure'] else 'NO'}")
    print(f"Known LLaVA behavior: {example['actual_llava_behavior']}")
    print("-" * 80)
    
    orig_response = generate_response(orig_img, example['original_question'])
    pert_response = generate_response(pert_img, example['perturbed_question'])
    
    # Display images side by side
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    axes[0].imshow(orig_img)
    axes[0].set_title("Original Image", fontsize=14, fontweight='bold')
    axes[0].axis('off')
    
    axes[1].imshow(pert_img)
    axes[1].set_title(f"Perturbed Image ({example['perturbation_type']})", fontsize=14, fontweight='bold')
    axes[1].axis('off')
    
    # Add colored border for failure cases
    if example['expected_failure']:
        for ax in axes:
            rect = patches.Rectangle((0, 0), 1, 1, linewidth=5, edgecolor='red', 
                                   facecolor='none', transform=ax.transAxes)
            ax.add_patch(rect)
    
    plt.tight_layout()
    plt.show()
    
    # Display questions and responses
    print(f"\n📝 ORIGINAL QUESTION: {example['original_question']}")
    print(f"🤖 MODEL RESPONSE: {orig_response}")
    print(f"\n📝 PERTURBED QUESTION: {example['perturbed_question']}")
    print(f"🤖 MODEL RESPONSE: {pert_response}")
    
    # Consistency assessment
    print(f"\n{'='*80}")
    print("🎯 FESTA CONSISTENCY ASSESSMENT:")
    
    # Simple consistency check (you can implement more sophisticated methods)
    similar = orig_response.lower().strip() == pert_response.lower().strip()
    
    if similar:
        print("✅ CONSISTENT: Model provided identical responses")
        consistency_score = 1.0
    else:
        print("❌ INCONSISTENT: Model provided different responses")
        consistency_score = 0.0
    
    print(f"📊 Consistency Score: {consistency_score:.1f}")
    
    if example['expected_failure'] and consistency_score < 1.0:
        print("🎯 Expected failure confirmed - this perturbation typically causes inconsistencies")
    elif not example['expected_failure'] and consistency_score == 1.0:
        print("🎯 Expected success confirmed - model maintained consistency")
    
    print(f"{'='*80}\n")
    
    return {
        'example_id': example_id,
        'consistency_score': consistency_score,
        'expected_failure': example['expected_failure'],
        'original_response': orig_response,
        'perturbed_response': pert_response
    }

In [ ]:
# Run equivalent input assessment - SUCCESS CASE
result1 = display_equivalent_assessment("blink_val_Spatial_Relation_1_contrast1")

In [ ]:
# Run equivalent input assessment - FAILURE CASE  
result2 = display_equivalent_assessment("blink_val_Spatial_Relation_1_masking1")

## FESTA Assessment: Complementary Input Consistency

**Complementary inputs** have opposite/negated meanings. A consistent model should provide different responses.

In [ ]:
def display_complementary_assessment(example_id):
    # Find the example
    example = next(ex for ex in examples['complementary_examples'] if ex['id'] == example_id)
    
    # Load images
    orig_img = Image.open(f"examples/{example['original_image']}")
    neg_img = Image.open(f"examples/{example['negated_image']}")
    
    # Generate responses
    print(f"\n🔄 COMPLEMENTARY INPUT ASSESSMENT: {example['perturbation_type'].upper()}")
    print(f"Expected failure: {'YES' if example['expected_failure'] else 'NO'}")
    print(f"Known LLaVA behavior: {example['actual_llava_behavior']}")
    print("-" * 80)
    
    orig_response = generate_response(orig_img, example['original_question'])
    comp_response = generate_response(neg_img, example['complementary_question'])
    
    # Display images side by side
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    axes[0].imshow(orig_img)
    axes[0].set_title("Original Image", fontsize=14, fontweight='bold')
    axes[0].axis('off')
    
    axes[1].imshow(neg_img)
    axes[1].set_title(f"Negated Image ({example['perturbation_type']})", fontsize=14, fontweight='bold')
    axes[1].axis('off')
    
    # Add colored border for failure cases
    if example['expected_failure']:
        for ax in axes:
            rect = patches.Rectangle((0, 0), 1, 1, linewidth=5, edgecolor='orange', 
                                   facecolor='none', transform=ax.transAxes)
            ax.add_patch(rect)
    
    plt.tight_layout()
    plt.show()
    
    # Display questions and responses
    print(f"\n📝 ORIGINAL QUESTION: {example['original_question']}")
    print(f"🤖 MODEL RESPONSE: {orig_response}")
    print(f"\n📝 COMPLEMENTARY QUESTION: {example['complementary_question']}")
    print(f"🤖 MODEL RESPONSE: {comp_response}")
    
    # Consistency assessment  
    print(f"\n{'='*80}")
    print("🎯 FESTA CONSISTENCY ASSESSMENT:")
    
    # For complementary inputs, we want DIFFERENT responses
    different = orig_response.lower().strip() != comp_response.lower().strip()
    
    if different:
        print("✅ CONSISTENT: Model provided different responses for opposite meanings")
        consistency_score = 1.0
    else:
        print("❌ INCONSISTENT: Model provided same response for opposite meanings")
        consistency_score = 0.0
    
    print(f"📊 Consistency Score: {consistency_score:.1f}")
    
    if example['expected_failure'] and consistency_score < 1.0:
        print("🎯 Expected failure confirmed - model struggles with negated/opposite concepts")
    elif not example['expected_failure'] and consistency_score == 1.0:
        print("🎯 Expected success confirmed - model correctly handled opposite meanings")
    
    print(f"{'='*80}\n")
    
    return {
        'example_id': example_id,
        'consistency_score': consistency_score,
        'expected_failure': example['expected_failure'],
        'original_response': orig_response,
        'complementary_response': comp_response
    }

In [ ]:
# Run complementary input assessment - FAILURE CASE
result3 = display_complementary_assessment("blink_val_Spatial_Relation_1_negated_contrast1")

## Summary: FESTA Assessment Results

The examples above demonstrate real failure cases from the BLINK dataset where LVLMs show inconsistencies.

In [ ]:
# Collect all results
results = [result1, result2, result3]

print("📊 FESTA ASSESSMENT SUMMARY")
print("=" * 50)

total_consistency = sum(r['consistency_score'] for r in results)
avg_consistency = total_consistency / len(results)

print(f"Overall Consistency Score: {avg_consistency:.2f} / 1.00")
print(f"Total Assessments: {len(results)}")
print(f"Consistent Responses: {int(total_consistency)}")
print(f"Inconsistent Responses: {len(results) - int(total_consistency)}")

print("\n📈 Breakdown by Assessment Type:")

equiv_results = [r for r in results if 'contrast1' in r['example_id'] or 'masking1' in r['example_id']]
comp_results = [r for r in results if 'negated' in r['example_id']]

if equiv_results:
    equiv_avg = sum(r['consistency_score'] for r in equiv_results) / len(equiv_results)
    print(f"  • Equivalent Input Consistency: {equiv_avg:.2f}")

if comp_results:
    comp_avg = sum(r['consistency_score'] for r in comp_results) / len(comp_results)
    print(f"  • Complementary Input Consistency: {comp_avg:.2f}")

print("\n🎯 Key Findings:")
print("  • Masking perturbations often cause equivalent input failures")
print("  • Negated questions frequently lead to complementary input failures")
print("  • Contrast perturbations tend to be more robust")

print("\n📚 About FESTA:")
print("  • Framework for systematic LVLM consistency assessment")
print("  • Identifies failure points using equivalent & complementary inputs")
print("  • Helps improve model robustness and reliability")
print("\n🔗 Learn more: https://github.com/iiscleap/mllm-uncertainty-estimation/")

## Interactive Section: Test Your Own Images

Upload your own image and test the model's consistency with equivalent questions!

In [ ]:
from google.colab import files
import io

def test_custom_image():
    print("Upload an image to test LVLM consistency:")
    uploaded = files.upload()
    
    if uploaded:
        # Get the first uploaded file
        filename = list(uploaded.keys())[0]
        image = Image.open(io.BytesIO(uploaded[filename]))
        
        # Display the image
        plt.figure(figsize=(8, 6))
        plt.imshow(image)
        plt.title("Your Uploaded Image", fontsize=16, fontweight='bold')
        plt.axis('off')
        plt.show()
        
        # Get user questions
        print("\nEnter two equivalent questions about this image:")
        q1 = input("Question 1: ")
        q2 = input("Question 2 (equivalent meaning): ")
        
        # Generate responses
        print("\n🤖 Generating responses...")
        r1 = generate_response(image, q1)
        r2 = generate_response(image, q2)
        
        # Display results
        print(f"\n📝 Question 1: {q1}")
        print(f"🤖 Response 1: {r1}")
        print(f"\n📝 Question 2: {q2}")
        print(f"🤖 Response 2: {r2}")
        
        # Assess consistency
        consistent = r1.lower().strip() == r2.lower().strip()
        print(f"\n🎯 FESTA Assessment: {'CONSISTENT' if consistent else 'INCONSISTENT'}")
        
        return {
            'questions': [q1, q2],
            'responses': [r1, r2],
            'consistent': consistent
        }
    else:
        print("No image uploaded.")
        return None

# Uncomment the line below to test with your own image
# custom_result = test_custom_image()

---

## Conclusion

This demo showed how the **FESTA framework** systematically assesses LVLM consistency using real dataset examples. The framework revealed:

- **Equivalent Input Failures**: Masking perturbations often cause inconsistencies
- **Complementary Input Failures**: Models struggle with negated/opposite concepts
- **Robustness Variations**: Different perturbation types have varying impact

FESTA helps identify these failure points to improve LVLM reliability and robustness.

**Next Steps**: 
- Explore the full FESTA framework on GitHub
- Try more examples from BLINK and VSR datasets  
- Implement consistency improvements based on identified failures

🔗 **Resources**:
- Paper: https://arxiv.org/pdf/2410.23499.pdf
- Code: https://github.com/iiscleap/mllm-uncertainty-estimation/